In [1]:
import os
import openai
import sys
import yaml

# change to your token
with open('./token.yaml', 'r') as file:
    keys = yaml.safe_load(file)
    
openai.api_key = keys["openai_key"]


- Baseline

  將第一次作業的 QA 檢索聊天機器人，重新透過 LangChain 進行實作


- Advanced（Optional）

  延伸 Basline 的內容，測試看看不同的 chunk 大小，對於生成結果的影響

## Document loading

In [2]:
from langchain.document_loaders import TextLoader

In [3]:
data_path = "../語料庫/chinesedora.txt"

loader = TextLoader(data_path)
pages = loader.load()

In [4]:
pages

[Document(metadata={'source': '../語料庫/chinesedora.txt'}, page_content='指引天使／引路天使是一個道具可以用來：它可以告訴你最好的路，不至於在路上出意外。\n雲朵凝固瓦斯／凝雲噴霧是一個道具可以用來：使用這個瓦斯的話，雲會凝固，變得可以奔走在雲上。\n交換繩是一個道具可以用來：兩人同時握住繩索的話，心還是一樣，但身體替換。\n室內旅行機是一個道具可以用來：可以顯示出各種地點的立體影像，但室內面積並不會隨之更動。會讓自己的家有旅行的味道。\n落難神明／天神機械人是一個道具可以用來：是一個難看老機器人。不過，幫助他的話，可以滿足3個請求。\n夢風鈴是一個道具可以用來：使用這個風鈴，就可以將睡覺的人叫起來(他們是夢遊狀態)，並且任憑差遣。\nＮＳ徽章／南北極徽章是一個道具可以用來：只要將自己與另外一個人貼上寫有「Ｎ」或「Ｓ」的徽章，就會變的像磁鐵一樣，同極相斥，異極相吸。\n阿拉丁神燈是一個道具可以用來：擦拭油燈的話，冒煙的機器人就會出現，不管什麼命令都聽。\n間諜工具組合／間諜套裝是一個道具可以用來：能用監視器看間諜人偶的「麥克風」和「」偵察的影像。\n聲音糖果／聲音糖是一個道具可以用來：只要利用這個錄製聲音，再給別人吃下去以後， 就可以得到被錄製聲音的特徵。\n進化退化放射線槍／進化退化光線槍是一個道具可以用來：被這光照射到，動物就可以進化(或是退化)。\n夢境電視／顯夢電視是一個道具可以用來：能除去別人看的夢的電視。\n算了算了棒／算啦棒是一個道具可以用來：用這個棒子將正在生氣的人的嘴巴堵住，並說「算了，算了」，那人氣就消了。\n助興樂團／增加氣氛樂團是一個道具可以用來：這兩個道具都是人型玩偶，都會播放音樂，讓氣氛變好；不同的是，只要讓小型插秧姑娘唱歌，就可以快速地完成插秧。\n四次元口袋／四度空間百寶袋是一個道具可以用來：哆啦A夢的道具都是放在這裡面的。四次元空間是個沒有大小的空間，要多大有多大。\n美食桌巾／美食布是一個道具可以用來：它可以幫你做出好吃的料理。無論甚麼料理都可以變出來。\n邱比特之箭／愛神之箭是一個道具可以用來：被箭射中的人，會對射箭的人產生好感。\n感動麥克風／感動擴音器是一個道具可以用來：使用這個麥克風，發出的聲音就會讓人感動不已。\n桃太郎丸子／桃太郎飯糰是一個道具可以用來

## Document Splitting

In [5]:

from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

In [6]:
c_splitter = CharacterTextSplitter(
    chunk_size=450,
    chunk_overlap=0,
    separator="\n"
)
texts_c_chunks = c_splitter.split_documents(pages)

In [34]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=80,
    chunk_overlap=0,
    separators=["\n\n", "\n", " ", ""])
texts_r_chunks = r_splitter.split_documents(pages)

In [36]:
texts_r_chunks

[Document(metadata={'source': '../語料庫/chinesedora.txt'}, page_content='指引天使／引路天使是一個道具可以用來：它可以告訴你最好的路，不至於在路上出意外。'),
 Document(metadata={'source': '../語料庫/chinesedora.txt'}, page_content='雲朵凝固瓦斯／凝雲噴霧是一個道具可以用來：使用這個瓦斯的話，雲會凝固，變得可以奔走在雲上。'),
 Document(metadata={'source': '../語料庫/chinesedora.txt'}, page_content='交換繩是一個道具可以用來：兩人同時握住繩索的話，心還是一樣，但身體替換。'),
 Document(metadata={'source': '../語料庫/chinesedora.txt'}, page_content='室內旅行機是一個道具可以用來：可以顯示出各種地點的立體影像，但室內面積並不會隨之更動。會讓自己的家有旅行的味道。'),
 Document(metadata={'source': '../語料庫/chinesedora.txt'}, page_content='落難神明／天神機械人是一個道具可以用來：是一個難看老機器人。不過，幫助他的話，可以滿足3個請求。'),
 Document(metadata={'source': '../語料庫/chinesedora.txt'}, page_content='夢風鈴是一個道具可以用來：使用這個風鈴，就可以將睡覺的人叫起來(他們是夢遊狀態)，並且任憑差遣。'),
 Document(metadata={'source': '../語料庫/chinesedora.txt'}, page_content='ＮＳ徽章／南北極徽章是一個道具可以用來：只要將自己與另外一個人貼上寫有「Ｎ」或「Ｓ」的徽章，就會變的像磁鐵一樣，同極相斥，異極相吸。'),
 Document(metadata={'source': '../語料庫/chinesedora.txt'}, page_content='阿拉丁神燈是一個道具可以用來：擦拭油燈的話，冒煙的機器人就會出現，不管什麼命令都聽。'),
 Document(metadata=

## Vectorstores and Embeddings

In [10]:
from langchain_huggingface import HuggingFaceEmbeddings

In [37]:
# 初始化 HuggingFace 的嵌入模型
embeddings_model = HuggingFaceEmbeddings(model_name="infgrad/stella-base-zh-v3-1792d",
                                         model_kwargs={"device": "cpu"},
                                         encode_kwargs={"normalize_embeddings": True})


Some weights of BertModel were not initialized from the model checkpoint at infgrad/stella-base-zh-v3-1792d and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
d:\JZH\國泰\proj\llama_club\.venv\lib\site-packages\sentence_transformers\models\Dense.py:89: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowliste

In [38]:
# 將文本塊轉換為嵌入
embeddings = [embeddings_model.embed_query(chunk.page_content) for chunk in texts_r_chunks]

In [39]:
# from langchain.embeddings.openai import OpenAIEmbeddings
# embeddings = OpenAIEmbeddings(openai_api_key=keys["openai_key"]) # model default = text-embedding-ada-002

In [40]:
from langchain.vectorstores import Chroma

In [41]:
# load it into Chroma
db = Chroma.from_documents(texts_r_chunks, embeddings_model)

In [42]:
# query it
query = "引路天使的功能是甚麼?"
docs = db.similarity_search_with_score(query)

In [43]:
docs

[(Document(metadata={'source': '../語料庫/chinesedora.txt'}, page_content='指引天使／引路天使是一個道具可以用來：它可以告訴你最好的路，不至於在路上出意外。'),
  0.29215529561042786),
 (Document(metadata={'source': '../語料庫/chinesedora.txt'}, page_content='指引天使／引路天使是一個道具可以用來：它可以告訴你最好的路，不至於在路上出意外。\n雲朵凝固瓦斯／凝雲噴霧是一個道具可以用來：使用這個瓦斯的話，雲會凝固，變得可以奔走在雲上。'),
  0.5349820852279663),
 (Document(metadata={'source': '../語料庫/chinesedora.txt'}, page_content='上的行星，或者是電腦地圖上沒有的地區。是很常用的道具，使用率位列第三，僅次於竹蜻蜓、時光機。'),
  0.883296012878418),
 (Document(metadata={'source': '../語料庫/chinesedora.txt'}, page_content='翻譯蒟蒻／翻譯糕是一個道具可以用來：吃了這個蒟蒻，任何人說的任何話，我們都可以聽懂、而且可以溝通。\n縮小燈／縮小電筒是一個道具可以用來：被光照射到，東西會變小。'),
  0.9327221512794495)]

## Retrieval

In [22]:
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

In [44]:
# 告訴retriever，要萃取出什麼資訊
metadata_field_info = [
    AttributeInfo(
        name="道具名稱",
        description="道具名稱",
        type="string",
    ),
    AttributeInfo(
        name="功能",
        description="這個道具的功能是甚麼",
        type="string",
    ),
]

In [53]:
document_content_description = "道具說明書" # 告訴retriever, ducument的內文代表什麼意思
llm = OpenAI(model='gpt-3.5-turbo-instruct', temperature=0, openai_api_key=keys["openai_key"])


In [46]:
retriever = SelfQueryRetriever.from_llm(
    llm, 
    db,
    document_content_description,
    metadata_field_info,
    verbose=True
)

In [47]:
question = "引路天使的功能是甚麼?"

In [48]:
docs = retriever.get_relevant_documents(question)

In [49]:
for d in docs:
    print(d.page_content)
    print('\n')

指引天使／引路天使是一個道具可以用來：它可以告訴你最好的路，不至於在路上出意外。


指引天使／引路天使是一個道具可以用來：它可以告訴你最好的路，不至於在路上出意外。
雲朵凝固瓦斯／凝雲噴霧是一個道具可以用來：使用這個瓦斯的話，雲會凝固，變得可以奔走在雲上。


竹蜻蜓是一個道具可以用來：雖然很委屈的排到第 3


落難神明／天神機械人是一個道具可以用來：是一個難看老機器人。不過，幫助他的話，可以滿足3個請求。
夢風鈴是一個道具可以用來：使用這個風鈴，就可以將睡覺的人叫起來(他們是夢遊狀態)，並且任憑差遣。




## Question Answering

In [51]:
import openai
# from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain


In [54]:
# 創建一個模板
template = """
問題: {question}
資料來源: {content}
"""

# 創建PromptTemplate
prompt = PromptTemplate(template=template, input_variables=["question", "content"])

# 創建LLMChain
llm = OpenAI(model='gpt-3.5-turbo-instruct', temperature=0, openai_api_key=keys["openai_key"])
chain = LLMChain(llm=llm, prompt=prompt)

# 從列表中檢索第0個內容
content = docs[0].page_content


d:\JZH\國泰\proj\llama_club\.venv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [55]:
# 問題是固定的字串
question = "引路天使的功能是甚麼?"

In [56]:
# 使用鏈條生成回答
response = chain.run({"question": question, "content": content})

print(response)

d:\JZH\國泰\proj\llama_club\.venv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(



引路天使的功能是指引人們走上最好的路徑，避免在路上遇到意外或迷路。它可以提供路線規劃、交通資訊、地圖導航等功能，讓人們能夠更快速、安全地到達目的地。有些引路天使還具備語音導航功能，可以在行走中提供指示，讓人們不用一直看地圖或手機，更加方便使用。引路天使也可以提供附近的景點、餐廳、住宿等資訊，讓人們能夠更加便利地計劃旅程。總的來說，引路天使的功能就是幫助人們找到最佳路線，讓旅行更加順利、安全、舒
